In [21]:
import numpy as np
import zarr

from src.common.files import get_raw_paths, get_processed_path
from src.visualization.render_mp4 import unpickle_data, pickle_data

from scipy.spatial.transform import Rotation as R

from tqdm import tqdm

In [2]:
raw_paths = get_raw_paths(
    environment="real",
    demo_outcome="success",
    demo_source="teleop",
    randomness="low",
    task="place_shade",
)

len(raw_paths)

Found the following paths:
    /data/scratch-oc40/pulkitag/ankile/furniture-data/raw/real/place_shade/teleop/low/**/success/*.pkl*


22

### Convert the data to be stored with delta actions in the pickle file

In [18]:
for raw_path in tqdm(raw_paths):
    data = unpickle_data(raw_path)

    actions = np.array(data["actions"])
    robot_state = np.stack([o["robot_state"] for o in data["observations"]])

    robot_pos, robot_quat = robot_state[:, :3], robot_state[:, 3:]
    action_pos, action_quat, action_gripper = actions[:, :3], actions[:, 3:7], actions[:, -1:]

    # Calclate what the delta action is
    action_delta_pos = action_pos - robot_pos

    # Calculate the delta quaternion
    robot_r = R.from_quat(robot_quat)
    action_r = R.from_quat(action_quat)
    action_delta_r = robot_r.inv() * action_r
    action_delta_quat = action_delta_r.as_quat()

    # Concatenate the delta action
    action_delta = np.concatenate([action_delta_pos, action_delta_quat, action_gripper], axis=-1)

    data["actions"] = action_delta
    pickle_data(data, raw_path)

100%|██████████| 22/22 [04:01<00:00, 10.97s/it]


In [20]:
zarr_path = get_processed_path(
    environment="real",
    demo_outcome="success",
    demo_source="teleop",
    randomness="low",
    task="place_shade",
)

zarr_path

PosixPath('/data/scratch/ankile/furniture-data/processed/real/place_shade/teleop/low/success.zarr')

In [23]:
z = zarr.open(zarr_path)

list(z.keys())

['action',
 'augment_states',
 'color_image1',
 'color_image2',
 'critical_state_id',
 'episode_ends',
 'failure_idx',
 'furniture',
 'parts_poses',
 'pickle_file',
 'reward',
 'robot_state',
 'skill',
 'success']

In [27]:
# Print ranges of the robot_state and actions
robot_state = z["robot_state"][:]
actions = z["action/pos"][:]

robot_state.shape, actions.shape

((6162, 9), (6162, 10))

In [33]:
print(robot_state.min(axis=0).round(3))
print(robot_state.max(axis=0).round(3))

[ 0.42  -0.241  0.181 -0.517 -1.    -0.166 -1.    -0.96  -0.237]
[ 0.668  0.292  0.536  0.961 -0.258  0.257 -0.242  0.516  0.26 ]


In [35]:
print(actions.min(axis=0).round(3))
print(actions.max(axis=0).round(3))

[ 0.415 -0.243  0.169 -0.529 -1.    -0.172 -1.    -0.961 -0.249 -1.   ]
[ 0.673  0.292  0.536  0.959 -0.262  0.252 -0.243  0.528  0.272  1.   ]
